In [14]:
import requests
import json
import pandas as pd
import random

url = "https://dapi.kakao.com/v2/local/search/keyword.json"
apiKey = "daa75b03ddbcfe35d4b5e51a7f1c86b0"

In [15]:
def getLocalData(query, page, x=None, y=None, radius=None):
    headers = {"Authorization": "KakaoAK " + apiKey}

    if x and y and radius:
        print("INFO : 중심점 좌표 입력됨")
        params = {
            'query': query, 
            'page': page, 
            'size': 15,
            'x': x,
            'y': y,
            'radius': radius
        }
    else:
        print("INFO : 중심점 좌표 입력 안됨")
        params = {'query': query, 'page': page, 'size': 15}

    request = requests.get(url, params=params, headers=headers).json()

    return request

In [16]:
def parsingStoreInfo(place):
    placeName = place["place_name"]
    placeUrl = place["place_url"]
    placeAddress = place["road_address_name"]
    placeDistance = place["distance"]
    placePhone = place["phone"]
    placeGroupCode = place["category_group_code"]
    placeCategoryName = place["category_name"]

    return (
        placeName,
        placeUrl,
        placeAddress,
        placeDistance,
        placePhone,
        placeGroupCode,
        placeCategoryName
    )

In [19]:
def gatheringAroundStoreInfo(query, lastPage, coordinateX, coordinateY, radius):
    storeList = pd.DataFrame(
        columns=["업체명", "카테고리코드", "카테고리", "전화번호", "홈페이지", "거리"]
    )

    for cnt in range(1, lastPage + 1, 1):
        print(cnt, "페이지 읽는중", end="\r")
        places = getLocalData(query, cnt, coordinateX, coordinateY, radius=700)

        meta = places["meta"]
        places = places["documents"]
        for place in places:
            (
                placeName,
                placeUrl,
                placeAddress,
                placeDistance,
                placePhone,
                placeGroupCode,
                placeCategoryName
            ) = parsingStoreInfo(place)
            storeList.loc[len(storeList)] = [
                placeName,
                placeGroupCode,
                placeCategoryName,
                placePhone,
                placeUrl,
                placeDistance
            ]

        if meta["is_end"]:
            print("마지막 페이지입니다.")
            break

        if cnt == lastPage:
            print("최대 페이지 한도 도달!")

    return storeList

In [21]:
lastPage = 10

places = getLocalData("대륭테크노타운3차", 1)
coordinateX, coordinateY = places["documents"][0]["x"], places["documents"][0]["y"]
print("좌표 : ", coordinateX, coordinateY)

storeList = gatheringAroundStoreInfo(
    "가산디지털단지맛집", 10, coordinateX, coordinateY, radius=700
)
storeList["카테고리"] = storeList["카테고리"].apply(lambda x: x.split(">")[1].strip())

print("검색된 상점의 카테고리에는", storeList["카테고리"].unique(), "가 있습니다")
preferCategory = input("이중 특별히 원하는 카테고리가 있으신가요? 없으면 엔터")

if preferCategory:
    idxList = storeList[storeList["카테고리"] == preferCategory].index
    storeInfo = storeList.loc[random.choice(idxList)]
else:
    storeInfo = storeList.loc[random.choice(storeList.index)]

print(f"오늘 {storeInfo['업체명']} 는 어떠세요?")
print(f"카테고리는 {storeInfo['카테고리']}에 속하고 거리는 {storeInfo['거리']}m 떨어져 있습니다.")
print(f"전화번호는 {storeInfo['전화번호']}이고, 홈페이지는 {storeInfo['홈페이지']} 입니다.")

INFO : 중심점 좌표 입력 안됨
좌표 :  126.87900718112 37.4778874758038
INFO : 중심점 좌표 입력됨
INFO : 중심점 좌표 입력됨
INFO : 중심점 좌표 입력됨
마지막 페이지입니다.
검색된 상점의 카테고리에는 ['뷔페' '한식' '카페' '분식' '술집' '패스트푸드' '아시아음식' '샤브샤브' '일식' '양식' '중식' '치킨' '샐러드'
 '구내식당'] 가 있습니다
오늘 옛날잔치국수전문점 는 어떠세요?
카테고리는 한식에 속하고 거리는 573m 떨어져 있습니다.
전화번호는 02-2113-7790이고, 홈페이지는 http://place.map.kakao.com/1541461620 입니다.


In [4]:
places['meta']

{'is_end': True,
 'pageable_count': 45,
 'same_name': {'keyword': '맛집', 'region': [], 'selected_region': '서울 마포구 공덕동'},
 'total_count': 13849}

In [5]:
places['documents']

[{'address_name': '서울 마포구 공덕동 471',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 양식',
  'distance': '',
  'id': '15110084',
  'phone': '02-3275-2323',
  'place_name': '탑클라우드23',
  'place_url': 'http://place.map.kakao.com/15110084',
  'road_address_name': '서울 마포구 백범로 192',
  'x': '126.951755204368',
  'y': '37.5433524952541'},
 {'address_name': '서울 마포구 공덕동 256-10',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 한식 > 육류,고기 > 족발,보쌈',
  'distance': '',
  'id': '16048613',
  'phone': '02-703-5141',
  'place_name': '장수족발',
  'place_url': 'http://place.map.kakao.com/16048613',
  'road_address_name': '서울 마포구 만리재로 19',
  'x': '126.95332605266819',
  'y': '37.54475688351382'},
 {'address_name': '서울 마포구 공덕동 477',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 간식 > 제과,베이커리',
  'distance': '',
  'id': '658344796',
  'phone': '',
  'place_name': '수더분',
  'place_url': 'ht

In [6]:
places['documents'][0]

{'address_name': '서울 마포구 공덕동 471',
 'category_group_code': 'FD6',
 'category_group_name': '음식점',
 'category_name': '음식점 > 양식',
 'distance': '',
 'id': '15110084',
 'phone': '02-3275-2323',
 'place_name': '탑클라우드23',
 'place_url': 'http://place.map.kakao.com/15110084',
 'road_address_name': '서울 마포구 백범로 192',
 'x': '126.951755204368',
 'y': '37.5433524952541'}